# This set of functions can be used to load the ARXIV data file and send it to the AWS queue

sciml_data_arxiv.p  is  the set of original arxiv rss items.  a subset of these were use for training.  the url is https://1drv.ms/u/s!AkRG9Zk_IOUagrFxuO3yxvluvwQ1uA This is a OneDrive link so you will need to go there on your browser and go to the "download" tab at the top and download the file.   perhaps in "/tmp"

sciml_data_arxiv_new_9_28_15.p  "newly" arrived arxiv items (not in the above) it is at https://1drv.ms/u/s!AkRG9Zk_IOUagrFvwrNNB8Jgc1ARcA   you can also use this one.  it does not contain anything  that was used for training.



In [1]:
import pickle
import json
import boto3

The following function just loads the pickled data file  and extracts the titles, sitenames (arxiv designations) and abstracts

In [2]:
def load_docs(path, name):
    filename = path+name+".p"
    fileobj = open(filename, "rb")
    z = fileobj.read()
    lst = pickle.loads(str(z))
    titles = []
    sitenames = []
    abstracts = []
    for i in range(0, len(lst)):
        titles.extend([lst[i][0]])
        sitenames.extend([lst[i][1]])
        abstracts.extend([lst[i][2]])
        
    print "done loading "+filename
    return abstracts, sitenames, titles

this call loads three arrays: abstracts: the paper abstract,  sites: the arxiv designation, and the title

In [3]:
abstracts, sites, titles = load_docs("/tmp/", "sciml_data_arxiv")

done loading /tmp/sciml_data_arxiv.p


let's take a peek at one of them.  how about number 1301

In [4]:
print abstracts[1301]
print sites[1301]
print titles[1301]

Cancer cells utilize large amounts of ATP to sustain growth, relying primarily on non-oxidative, fermentative pathways for its production. In many types of cancers this leads, even in the presence of oxygen, to the secretion of carbon equivalents (usually in the form of lactate) in the cell's surroundings, a feature known as the Warburg effect. While the molecular basis of this phenomenon are still to be elucidated, it is clear that the spilling of energy resources contributes to creating a peculiar microenvironment for tumors, possibly characterized by a degree of toxicity. This suggests that mechanisms for recycling the fermentation products (e.g. a lactate shuttle) may be active, effectively inducing a mutually beneficial metabolic coupling between aberrant and non-aberrant cells. Here we analyze this scenario through a large-scale in silico metabolic model of interacting human cells. By going beyond the cell-autonomous description, we show that elementary physico-chemical constrain

In [5]:
sqs = boto3.resource('sqs',
    region_name='us-west-2', 
 )


# now let's grab the queue
The queue "bookque" was previously created from the aws portal

In [6]:
queuename = 'bookque'
queue = sqs.get_queue_by_name(QueueName=queuename)
print(queue.url)
print(queue.attributes.get('DelaySeconds'))


https://us-west-2.queue.amazonaws.com/066301190734/bookque
5


In [7]:
import time

# send data to the queue
this loop grabs 100 papers (1350 through 1449) and pushes them to the queue.  These 100 were chosen as a random sample.  nothing special about them. 

In [14]:
t0 = time.time()
for i in range(1350,1450):
    abstract = abstracts[i]
    source = sites[i]
    title = titles[i]
    #print abstract
    print title
    queue.send_message(MessageBody='boto3', MessageAttributes ={
            'Title':{ 'StringValue': title,
                      'DataType': 'String'}, 
            'Source':{ 'StringValue': source,
                      'DataType': 'String'}, 
            'Abstract':{ 'StringValue': abstract,
                      'DataType': 'String'} 
        })
t1 = time.time()

Coping with Space Neophobia in Drosophila melanogaster: The Asymmetric   Dynamics of Crossing a Doorway to the Untrodden [q-bio.NC]
New Scaling Relation for Information Transfer in Biological Networks [q-bio.MN]
The Composition Of A Disrupted Extrasolar Planetesimal At SDSS   J0845+2257 (Ton 345) [astro-ph.EP]
Tip cell overtaking occurs as a side effect of sprouting in   computational models of angiogenesis [q-bio.CB]
Modeling the ballistic-to-diffusive transition in nematode motility   reveals low-dimensional behavioral variation across species [q-bio.NC]
Superconducting dark energy [gr-qc]
Equipartitions and a Distribution for Numbers: A Statistical Model for   Benford&#x27;s Law [physics.data-an]
Uncertainty analysis and composite hypothesis under the likelihood   paradigm [q-bio.QM]
The long-tail distribution function of mutations in bacteria [q-bio.PE]
Simultaneous regulation of cell size and chromosome replication in   bacteria [q-bio.CB]
A Jump Distance Distribution-based Bayesi

In [15]:
print 'elapsed time ='+str(t1-t0)

elapsed time =3.36903595924


# what was in the dynamoDB table after the process ended?
To see the output that appeared in the dynodb table look at BookTable-dynammodb-output.csv

Note: it took 3 seconds to fill the queue and if we look at the csv file and sort by timestame we see it took 26 seconds to do all the processing using 8 predictors.  